In [1]:
from datasets import load_dataset
from tqdm import tqdm
from random import shuffle
import json

HF_TOKEN = "<your-hf-token>"

all_rows = []

/home/t-gummavarun/.miniconda3/envs/fairseq/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
ds = load_dataset("Open-Orca/1million-gpt-4", trust_remote_code=True, token=HF_TOKEN)

for row in tqdm(ds["train"], total=len(ds)):
    sys_prompt = (
        row["system_prompt"].strip()
        if row["system_prompt"]
        else "You are a helpful AI assistant. Answer the following question, as per the given instructions."
    )
    user_prompt = row["question"].strip()
    assistant = row["response"].strip()

    obj = [
        {"role": "system", "content": sys_prompt},
        {"role": "user", "content": user_prompt},
        {"role": "assistant", "content": assistant},
    ]

    all_rows.append({"messages": obj})

994896it [00:30, 32466.35it/s]       


In [3]:
ds = load_dataset("PawanKrd/math-gpt-4o-200k", trust_remote_code=True, token=HF_TOKEN)

for row in tqdm(ds["train"], total=len(ds)):
    sys_prompt = "You are a helpful AI math assistant. Answer the following question, and provide a step-by-step logical solution."
    user_prompt = row["prompt"].strip()
    assistant = row["response"].strip()

    obj = [
        {"role": "system", "content": sys_prompt},
        {"role": "user", "content": user_prompt},
        {"role": "assistant", "content": assistant},
    ]

    all_rows.append({"messages": obj})

200035it [00:04, 45755.83it/s]       


In [4]:
ds = load_dataset("PawanKrd/gpt-4o-200k", trust_remote_code=True, token=HF_TOKEN)

for row in tqdm(ds["train"], total=len(ds)):
    sys_prompt = (
        row["system"].strip()
        if row["system"]
        else "You are a helpful AI assistant. Answer the following question, as per the given instructions."
    )
    user_prompt = row["prompt"].strip()
    assistant = row["response"].strip()

    obj = [
        {"role": "system", "content": sys_prompt},
        {"role": "user", "content": user_prompt},
        {"role": "assistant", "content": assistant},
    ]

    all_rows.append({"messages": obj})

200023it [00:04, 42383.27it/s]       


In [5]:
ds = load_dataset("llm-wizard/alpaca-gpt4-data", trust_remote_code=True, token=HF_TOKEN)

for row in tqdm(ds["train"], total=len(ds)):
    sys_prompt = "You are a helpful AI assistant. Answer the following question, as per the given instructions."
    user_prompt = (
        f"{row['instruction'].strip()}\n\n{row['input'].strip()}"
        if row["input"]
        else row["instruction"].strip()
    )
    assistant = row["output"].strip()

    obj = [
        {"role": "system", "content": sys_prompt},
        {"role": "user", "content": user_prompt},
        {"role": "assistant", "content": assistant},
    ]

    all_rows.append({"messages": obj})

52002it [00:01, 32819.42it/s]        


In [6]:
shuffle(all_rows)

with open("sft_data.jsonl", "w", encoding="utf-8") as f:
    for row in all_rows:
        f.write(json.dumps(row, ensure_ascii=False) + "\n")

print(f"Wrote {len(all_rows)} rows to sft_data.jsonl")

Wrote 1446956 rows to sft_data.jsonl


In [7]:
all_rows = []

ds = load_dataset("PawanKrd/dpo-gpt-4o", trust_remote_code=True, token=HF_TOKEN)

for row in tqdm(ds["train"], total=len(ds)):
    sys_prompt = (
        row["system"].strip()
        if row["system"]
        else "You are a helpful AI assistant. Answer the following question, as per the given instructions."
    )
    user_prompt = row["prompt"].strip()
    chosen = row["chosen"].strip()
    rejected = row["rejected"].strip()


    chosen_obj = [
        {"role": "system", "content": sys_prompt},
        {"role": "user", "content": user_prompt},
        {"role": "assistant", "content": chosen}
    ]

    rejected_obj = [
        {"role": "system", "content": sys_prompt},
        {"role": "user", "content": user_prompt},
        {"role": "assistant", "content": rejected}
    ]
    
    obj = {
        "chosen": chosen_obj,
        "rejected": rejected_obj
    }

    all_rows.append(obj)

12359it [00:00, 29863.70it/s]        


In [8]:
shuffle(all_rows)

with open("dpo_data.jsonl", "w", encoding="utf-8") as f:
    for row in all_rows:
        f.write(json.dumps(row, ensure_ascii=False) + "\n")

print(f"Wrote {len(all_rows)} rows to dpo_data.jsonl")

Wrote 12359 rows to dpo_data.jsonl
